# 0. Load Data

In [1]:
import math 
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Isolation Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# VAE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics

In [2]:
cell_ids = pd.read_csv("data/cells/cell_ids.csv",header=None)
cell_ids = [x for lst in cell_ids.values.tolist() for x in lst]
cell_ids

[2.2265366483183206e+17,
 7.315874467898523e+16,
 9.10621795573706e+16,
 3.8447326973958944e+17,
 4.262775046883192e+17]

In [3]:
cells = []
for cell_id in cell_ids:
    file_name = "cell_"+str(cell_id)+'.csv'
    cells.append(pd.read_csv(f"data/cells/{file_name}"))

In [4]:
cells[0].head()

,index,cell_id,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,PDCP_SDU_Volume_DL,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,Intra_eNB_HO_SR_total_RATIO,...,DCR_LTE_RATIO,CSSR_LTE_RATIO,LTE_INTER_ENODEB_HOSR_RATIO,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,DL_THROUGHPUT_RATIO,E_RAB_DR_RATIO,HOUR
0,2021-05-09 00:00:00,2.226537e+17,3.779737e+10,3.947172e+09,15.0,3.779737e+10,4727.0,0.809859,0.992427,0.809859,...,0.001761,0.996041,0.400000,0.770642,0.963636,0.963636,0.400000,82.550417,0.001761,0
1,2021-05-09 01:00:00,2.226537e+17,3.684898e+10,4.088752e+09,6.0,3.684898e+10,3076.0,0.886792,0.993288,0.886792,...,0.002468,0.995465,0.500000,0.842105,1.000000,1.000000,0.500000,80.531811,0.002468,1
2,2021-05-09 02:00:00,2.226537e+17,3.292677e+10,5.016897e+09,8.0,3.292677e+10,3501.0,0.938356,0.994664,0.938356,...,0.003077,0.996044,0.375000,0.931624,1.000000,0.966667,0.375000,71.885311,0.003077,2
3,2021-05-09 03:00:00,2.226537e+17,3.021547e+10,5.139107e+09,9.0,3.021547e+10,2275.0,0.860215,0.994819,0.860215,...,0.001721,0.995920,0.777778,0.816901,0.947368,0.947368,0.777778,65.977430,0.001721,3
4,2021-05-09 04:00:00,2.226537e+17,3.082176e+10,4.250716e+09,17.0,3.082176e+10,2178.0,0.840426,0.995952,0.840426,...,0.002213,0.995628,0.764706,0.794521,1.000000,1.000000,0.764706,67.324805,0.002213,4


# 1. Isolation Forest

In [5]:
model=IsolationForest(n_estimators=150, 
                      max_samples ='auto', 
                      max_features=1)

In [6]:
df_cell = cells[0].drop(['cell_id','index'],axis=1)
model.fit(df_cell)
# score_samples = - score  
IF_score = -1 * model.score_samples(df_cell)

In [ ]:
df_cell['IF_score'] = IF_score
df_cell['IF_score'].describe()

# 2. VAE

[keras - Variational AutoEncoder 官方代码范例(复杂)](https://keras.io/examples/generative/vae/)

[keras - AE/VAE 相对简单的代码构建](https://blog.keras.io/building-autoencoders-in-keras.html)

[keras - 中等程度AVE 但是有点乱](https://github.com/keras-team/keras/blob/2c8d1d03599cc03243bce8f07ed9c4a3d5f384f9/examples/variational_autoencoder.py)







[中文VAE理论+代码](https://blog.csdn.net/weixin_37737254/article/details/102920263)

[keras - 函数式API](https://keras.io/guides/functional_api/)

[keras - Conv1D](https://keras-zh.readthedocs.io/layers/convolutional/)


Split data

In [5]:
from sklearn.preprocessing import StandardScaler

df_cell = cells[0].drop(['cell_id','index'],axis=1)
df_cell = StandardScaler().fit_transform(df_cell)
df_cell = np.expand_dims(df_cell,axis=1)
x_train, x_test = train_test_split(df_cell)

Set parameters

Create sample layer

Build Encoder

Build Decoder

In [18]:
df_cell = cells[0].drop(['cell_id','index'],axis=1)
df_cell = StandardScaler().fit_transform(df_cell)
df_cell = np.expand_dims(df_cell,axis=1)
x_train, x_test = train_test_split(df_cell)


# encoder input
inputs = keras.Input(shape=(1,original_dim))
x = layers.Conv1D(filters=intermediate_dim, kernel_size=3, padding="same", strides=2, activation="relu")(inputs)
x = layers.Dropout(rate=0.2)(x)
x = layers.Flatten()(x)
h = layers.Dense(8,activation="relu")(x)

# the output of encoder
z_mean = layers.Dense(latent_dim, name="z_mean")(h)
z_log_sigma = layers.Dense(latent_dim, name="z_log_var")(h)
z = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 1, 24)]      0           []                               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 1, 32)        2336        ['input_7[0][0]']                
                                                                                                  
 dropout_7 (Dropout)            (None, 1, 32)        0           ['conv1d_2[0][0]']               
                                                                                                  
 flatten_1 (Flatten)            (None, 32)           0           ['dropout_7[0][0]']              
                                                                                            

In [19]:
strides = 2
intermediate_dim = 32

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(1*intermediate_dim, activation='relu')(latent_inputs)
x = layers.Reshape((1,intermediate_dim))(x)
x = layers.Conv1DTranspose(filters=intermediate_dim, kernel_size=3, padding="same", activation="relu")(x)
x = layers.Dropout(rate=0.2)(x)
outputs = layers.Conv1DTranspose(filters=original_dim, kernel_size=3, padding="same", activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

decoder.summary()


Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 2)]               0         
                                                                 
 dense_26 (Dense)            (None, 32)                96        
                                                                 
 reshape_2 (Reshape)         (None, 1, 32)             0         
                                                                 
 conv1d_transpose_4 (Conv1DT  (None, 1, 32)            3104      
 ranspose)                                                       
                                                                 
 dropout_8 (Dropout)         (None, 1, 32)             0         
                                                                 
 conv1d_transpose_5 (Conv1DT  (None, 1, 24)            2328      
 ranspose)                                                 

In [20]:
# Instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')
vae.compile(optimizer='adam',loss=keras.losses.mean_squared_error)

In [21]:
# Train model
vae.fit(x_train, x_train,
        epochs=100,
        batch_size=32,
        validation_data=(x_test, x_test))

Epoch 1/100
39/39 [==============================] - 1s 8ms/step - loss: 1.2323 - val_loss: 1.1236
Epoch 2/100
39/39 [==============================] - 0s 3ms/step - loss: 1.0850 - val_loss: 0.9591
Epoch 3/100
39/39 [==============================] - 0s 2ms/step - loss: 1.0060 - val_loss: 0.9327
Epoch 4/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9913 - val_loss: 0.9139
Epoch 5/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9641 - val_loss: 0.8748
Epoch 6/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9414 - val_loss: 0.8610
Epoch 7/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9284 - val_loss: 0.8522
Epoch 8/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9244 - val_loss: 0.8453
Epoch 9/100
39/39 [==============================] - 0s 3ms/step - loss: 0.9153 - val_loss: 0.8412
Epoch 10/100
39/39 [==============================] - 0s 2ms/step - loss: 0.9112 - val_loss: 0.8384
Epoch 11/

39/39 [==============================] - 0s 2ms/step - loss: 0.8344 - val_loss: 0.7737
Epoch 84/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8349 - val_loss: 0.7729
Epoch 85/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8349 - val_loss: 0.7729
Epoch 86/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8349 - val_loss: 0.7730
Epoch 87/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8338 - val_loss: 0.7728
Epoch 88/100
39/39 [==============================] - 0s 3ms/step - loss: 0.8327 - val_loss: 0.7758
Epoch 89/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8337 - val_loss: 0.7728
Epoch 90/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8350 - val_loss: 0.7727
Epoch 91/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8330 - val_loss: 0.7720
Epoch 92/100
39/39 [==============================] - 0s 2ms/step - loss: 0.8319 - val_loss: 0.7733
Epoch 93/100


In [ ]:
def anomalyPercent(model, df_test):
    """
    Calculate the percentage of anomalies in the testset
    
    """
    df_pred = df_test.copy()
    df_pred['anomaly'] = model.predict(df_test)
    pct  = (df_pred['anomaly']==-1).sum()/len(df_test)
    
#     # map 1 -> 0, -1 -> 1
#     reset_value = lambda x: 1 if x==-1 else 0
#     df_test['anomaly'] = df_test['anomaly'].map(reset_value)
    
    return pct, df_pred

In [ ]:
def anomaly_visualization(df):
    """
    Visualize anomalies in 2D/3D scatter plot
    
    INPUT
    @df : a dataframe where the prediction result is in the 'anomaly' column
    
    """
    index = df.index
    outlier_index = list(df[df['anomaly']==-1].index)
    
    # nomalize the metrics
    X = StandardScaler().fit_transform(df.drop('anomaly',axis=1))
    
    # recude the dimension for visualization
    X_3d = pd.DataFrame(PCA(3).fit_transform(X),index=index)
    X_3d_outliers = X_3d.reindex(outlier_index)
    X_2d = pd.DataFrame(PCA(2).fit_transform(X),index=index)
    X_2d_outliers = X_2d.reindex(outlier_index)
    
    # plot
    fig = plt.figure(figsize=(10,5))
    # 3D
    ax1 = fig.add_subplot(121,  projection='3d')
    ax1.scatter(X_3d.loc[:,0],X_3d.loc[:,1],X_3d.loc[:,2],
                s=4,lw=1,label="inliers",c="green",alpha=0.5)
    ax1.scatter(X_3d_outliers.loc[:,0], X_3d_outliers.loc[:,1], X_3d_outliers.loc[:,2],
               s=60,marker="x",lw=2,label="outliers",c="red")
    ax1.legend()
    ax1.set_title("Isolation Prediction (3D)")
    # 2D
    ax2 = fig.add_subplot(133)
    ax2.scatter(X_2d.loc[:,0],X_2d.loc[:,1],c="green",
                s=5,label="inliers",alpha=0.8)
    ax2.scatter(X_2d_outliers.iloc[:,0], X_2d_outliers.iloc[:,1],
                s=60,marker="x",lw=2,c='red',label="outliers")
    ax2.legend()
    ax2.set_title("Isolation Prediction (2D)")
    plt.show()
    

In [ ]:
for i in range(len(list_cells)):
    # get the dataframe of a certain cell
    df_c = list_cells[i].drop(['cell_id','index'], axis=1)
    
    # split train set and test set 
    df_train, df_test = train_test_split(df_c)
    # train model 
    model.fit(df_train)
    pct, df_pred = anomalyPercent(model, df_test)
    print("="*75)
    print(f"cell id: {cell_ids[i]}")
    print("Percentage of outliers: %.3f "%pct)
    anomaly_visualization(df_pred)
    print("\n\n")